## Trajectary Generation

This step format trajectory data to a table with columns:

`uid`, `date`, `traj`. `traj_site`, `traj_arr`,

where `traj_site` indicates a list of location codes that a user visits one day. `traj_arr` is the same trajectory with area codes instead. 

In [1]:
from ast import literal_eval
from collections import defaultdict
from datetime import datetime
import os

import pandas as pd

In [9]:
PROJECT_ROOT = '..'

# Data files
DATA_DIR = os.path.join(PROJECT_ROOT, 'data')
TRAJECTORY_FILES = [os.path.join(DATA_DIR, f) for f in [
    'SET2_P01.user3000.csv',
]]
AREA_FILE = os.path.join(DATA_DIR, 'SITE_ARR_LONLAT.CSV')
TRAJ_FILE = 'user3000.Jan.csv'

# Build files
BUILD_PATH = os.path.join(PROJECT_ROOT, 'build')
TRAJ_SEL_FILE = os.path.join(BUILD_PATH, 'selected.Jan.csv')
ARR_SEL_FILE = os.path.join(BUILD_PATH, 'selected.Jan.txt')

FORCE_REBUILD = True

In [5]:
def gen_traj(path, func_site2arr):
    """ Generate trajectory dataframe from original data file. """
    # Read csv from file.
    df = pd.read_csv(path,
                     names=['uid', 'date', 'traj_site'],
                     parse_dates=['date'],
                     date_parser=lambda x: pd.datetime.strptime(x, '%Y-%m-%d %H:%M:%S').date())
    # Get trajectory per user per data and remove a record if its traj_site is the same
    # as the previous record.
    def to_traj(x):
        traj = []
        for v in x:
            if not traj or traj[-1] != v:
                traj += v,
        return tuple(traj)
    df = df.groupby(['uid', 'date']).aggregate(to_traj)
    # Append a column that indicate the trajectary of area_id.
    df['traj_arr'] = df.traj_site.apply(lambda traj: tuple(map(func_site2arr, traj)))
    return df

def read_traj(path):
    """ Read generated trajectory datafrme for file. """
    df = pd.read_csv(TRAJ_FILE, index_col=[0,1], parse_dates=['date'])
    df['traj_site'] = df['traj_site'].apply(lambda x: literal_eval(x))
    df['traj_arr'] = df['traj_arr'].apply(lambda x: literal_eval(x))
    return df

if FORCE_REBUILD or not os.path.exists(TRAJ_FILE):
    df_arr = pd.read_csv(AREA_FILE, index_col=[0])
    DF_TRAJ = gen_traj(TRAJECTORY_FILES[0], lambda site_id: df_arr.arr_id[site_id])
    DF_TRAJ.to_csv(TRAJ_FILE)
else:
    DF_TRAJ = read_traj(TRAJ_FILE)
    
DF_TRAJ

traj_site  \
uid  date                                                            
1    2013-01-07                               (461, 454, 327, 323)   
     2013-01-08                                             (323,)   
     2013-01-09                          (323, 318, 323, 318, 323)   
     2013-01-10                                             (323,)   
     2013-01-11                               (323, 330, 318, 323)   
     2013-01-12                                             (323,)   
     2013-01-13                                             (323,)   
     2013-01-14                               (314, 312, 314, 323)   
     2013-01-15                                    (323, 314, 323)   
     2013-01-16                          (323, 314, 312, 314, 323)   
     2013-01-17                                    (318, 314, 323)   
     2013-01-18                (323, 314, 318, 314, 318, 308, 323)   
     2013-01-19                               (323, 314, 318, 314)   
     2013-01-20                                             (318,)   
2    2013-01-07  (272, 264, 293, 277, 304, 324, 341, 335, 354, ...   
     2013-01-08  (272, 264, 277, 412, 413, 412, 407, 413, 408, ...   
     2013-01-09  (272, 264, 277, 264, 311, 319, 419, 407, 413, ...   
     2013-01-10  (272, 277, 264, 277, 264, 419, 413, 412, 413, ...   
     2013-01-11  (277, 408, 394, 408, 475, 471, 480, 460, 471, ...   
     2013-01-12  (272, 264, 408, 473, 475, 473, 475, 472, 477, ...   
     2013-01-13      (272, 269, 209, 272, 269, 272, 260, 279, 290)   
     2013-01-14  (272, 355, 457, 475, 480, 475, 471, 475, 473, ...   
     2013-01-15  (475, 473, 475, 473, 471, 467, 471, 460, 471, ...   
     2013-01-16  (475, 468, 471, 319, 464, 475, 473, 475, 471, ...   
     2013-01-17  (475, 473, 475, 473, 475, 443, 448, 419, 416, ...   
     2013-01-18  (464, 475, 474, 476, 475, 473, 475, 476, 475, ...   
     2013-01-19  (475, 478, 476, 475, 473, 475, 476, 475, 473, ...   
     2013-01-20                (314, 272, 279, 269, 186, 279, 272)   
3    2013-01-07                                            (1123,)   
     2013-01-12                                 (1122, 1123, 1122)   
...                                                            ...   
2998 2013-01-17                                         (487, 481)   
     2013-01-18                                         (481, 487)   
     2013-01-19                                         (481, 487)   
     2013-01-20                               (481, 487, 479, 481)   
2999 2013-01-07                                             (603,)   
     2013-01-09                          (603, 605, 603, 605, 603)   
     2013-01-10                                    (607, 603, 605)   
     2013-01-11                                         (605, 603)   
     2013-01-12                                             (603,)   
     2013-01-13                                             (603,)   
     2013-01-14                                             (603,)   
     2013-01-15                                             (603,)   
     2013-01-16                                         (603, 605)   
     2013-01-17                                             (603,)   
     2013-01-19                                             (603,)   
     2013-01-20                                             (599,)   
3000 2013-01-07                                             (925,)   
     2013-01-08                                             (925,)   
     2013-01-09                                         (925, 940)   
     2013-01-10                                             (925,)   
     2013-01-11                                             (925,)   
     2013-01-12                                             (925,)   
     2013-01-13                                             (925,)   
     2013-01-14                                             (925,)   
     2013-01-15                                             (925,) 

## Area Frequency Analysis

In [11]:
os.mkdir(BUILD_PATH)

In [12]:
ARR2UIDS = defaultdict(set)
for idx, row in DF_TRAJ.iterrows():
    for arr in row.traj_arr:
        ARR2UIDS[arr].add(idx[0])

ARR_SORTED = sorted(ARR2UIDS, key=lambda x: len(ARR2UIDS[x]), reverse=True)
    
with open(ARR_SEL_FILE, 'w') as file:
    file.write(str(ARR_SORTED[0]))

#for arr in ARR_SORTED[:3]:
#    print('%3s: %s' % (arr, str(ARR2UIDS[arr])))

## Select the Most Frequent Area

In [7]:
frames = []
for uid in sorted(ARR2UIDS[ARR_SORTED[0]]):
    frames += DF_TRAJ.iloc[DF_TRAJ.index.get_level_values('uid') == uid,:],
DF_MOST_FREQ = pd.concat(frames)

DF_MOST_FREQ

traj_site  \
uid  date                                                            
6    2013-01-09                              (1092, 781, 719, 709)   
     2013-01-10           (362, 369, 256, 141, 174, 256, 174, 187)   
     2013-01-11                               (205, 174, 187, 256)   
     2013-01-12                               (434, 419, 423, 419)   
     2013-01-13           (434, 423, 419, 423, 419, 423, 419, 423)   
     2013-01-14                     (434, 179, 202, 179, 202, 179)   
     2013-01-15  (179, 202, 228, 202, 179, 202, 219, 228, 202, ...   
     2013-01-16                                            (1092,)   
     2013-01-17                                            (1092,)   
     2013-01-18                                       (1092, 1091)   
     2013-01-19                                            (1092,)   
     2013-01-20                                            (1092,)   
8    2013-01-07                          (232, 155, 140, 155, 140)   
     2013-01-08                                         (161, 155)   
     2013-01-09                                             (155,)   
     2013-01-11                                             (155,)   
     2013-01-13                                             (155,)   
     2013-01-14                                         (177, 155)   
     2013-01-17                                         (205, 155)   
     2013-01-18                                             (155,)   
     2013-01-19                                             (155,)   
     2013-01-20                                    (163, 185, 155)   
17   2013-01-07                          (171, 159, 141, 171, 168)   
     2013-01-08                                         (159, 141)   
     2013-01-09                                             (171,)   
     2013-01-13                                    (168, 171, 168)   
     2013-01-14                                             (640,)   
     2013-01-15                                    (670, 646, 664)   
     2013-01-16                                             (646,)   
     2013-01-17                                             (419,)   
...                                                            ...   
2980 2013-01-18                               (361, 367, 368, 367)   
     2013-01-19                               (367, 357, 368, 367)   
     2013-01-20                                             (367,)   
2983 2013-01-07                           (48, 45, 34, 48, 57, 34)   
     2013-01-08                                       (34, 57, 34)   
     2013-01-09                                       (57, 34, 45)   
     2013-01-10                                   (45, 37, 48, 34)   
     2013-01-11                               (34, 28, 57, 34, 48)   
     2013-01-12                                   (28, 57, 45, 34)   
     2013-01-13                                   (34, 48, 34, 65)   
     2013-01-14                                           (30, 45)   
     2013-01-15                                           (57, 38)   
     2013-01-17                                           (57, 34)   
     2013-01-18                                  (48, 38, 46, 216)   
     2013-01-19                                   (45, 38, 57, 34)   
     2013-01-20                                   (38, 49, 57, 38)   
2997 2013-01-07                                    (120, 124, 191)   
     2013-01-08                               (191, 137, 227, 124)   
     2013-01-09                               (124, 227, 128, 109)   
     2013-01-10                          (161, 131, 124, 150, 124)   
     2013-01-11                               (124, 139, 128, 124)   
     2013-01-12                                             (124,)   
     2013-01-13                               (124, 110, 124, 128)   
     2013-01-14      (191, 124, 109, 124, 147, 109, 124, 161, 124)   
     2013-01-15                               (124, 120, 128, 124) 

In [13]:
DF_MOST_FREQ.to_csv(TRAJ_SEL_FILE)